In [1]:
import warnings
import unittest
from prefixspan import PrefixSpan #modul khusus untuk prefixspan
import pandas as pd #baca file excel, csv
import nltk #untuk mengatur preprocessing
from nltk.tokenize import word_tokenize #proses tokenize dilakukan oleh nltk
from nltk.corpus import stopwords #stopwords dilakukan oleh nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #stemmer dilakukan oleh sastrawi, kenapa tidak menggunakan nltk? karena untuk proses stemming di nltk itu bagus untuk bhs inggris (jadi harus ditranslate ke inggris, setelah didapatkan hasil di translate lagi ke indonesia) sehingga kurang pas untuk proses di nltk
import re #untuk cleaning, jika butuh menghapus simbol2
import string #modul untuk huruf  dan angka saat cleaning
import os
import time
import resource
import matplotlib.pyplot as plt #untuk membuat gambar, seperti word cloud, yg tampil langsung diaplikasi


warnings.filterwarnings("ignore") #mematikan warning, karena warning bukan hanya muncul saat ada yang eror tetapi juga saat mengubah satu modul yang ada fitur maka warning akan muncul sehingga mengganggu
pd.options.mode.chained_assignment = None #ada beberapa operasi yg jika kita lakukan itu versi kedepan akan dihapus maka akan dikeluarkan warning sehingga tidak ditampilkan. 
nltk.download('stopwords') #untuk download library
nltk.download('punkt') #untuk download tanda henti 
factory = StemmerFactory()#mendefisi fungsi stemming
stemmer = factory.create_stemmer() #membuat stemming
listStopword =  set(stopwords.words('indonesian')) #load stopword, apa saja stopword nya dan menggunakan bahasa indonesia 

listtoberemoved = ['bengkulu', 'and', 'of', 'on','in','based','the','to','indonesia','thailand','from', 'for','berbasis','muara','bangkahulu','kota','di','untuk','lempuing'] #kata2 apa saja yg ingin dihapus karena tidak berpengaruh saat analisis

ModuleNotFoundError: ignored

In [ ]:
def memory_limit(): #untuk mengatur batasan memori
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (get_memory() * int(1024 / 2), hard))

def get_memory(): #untuk mengambil data memori
    with open('/proc/meminfo', 'r') as mem:
        free_memory = 0
        for i in mem:
            sline = i.split()
            if str(sline[0]) in ('MemFree:', 'Buffers:', 'Cached:'):
                free_memory += int(sline[1])
    return free_memory

In [ ]:
def read_data(x): #membaca data
    XL = pd.ExcelFile(os.path.join("data","data_mod.xlsx")) #membaca file
    sheet = XL.sheet_names[x] #dari file ada sheet apa saja, sheet tergantung urutan (pada python itu mulai dari 0-5, 0 itu LPPM)
    data = pd.read_excel(XL, sheet)
    data = data[~data['Topik'].isnull()] #memulai cleaning, jika tidak ada topik penelitian pada data tersebut maka akan ditampilkan lagi
    data['Tahun'] = data['Tahun'].astype(int) #data tahun diganti jadi integer 
    data = data.reset_index(drop=True) #direset index dan di drop, dan kemudian agar data yg tidak ada topik tadi terhapus, dan nomornya bisa kembali sesuai dengan urutan
    data = data[data.columns.tolist()[:-3]].join(data[data.columns.tolist()[-2:]]).join(data[data.columns.tolist()[-3]]) #untuk menggeser supaya kolom lebih urut saat data penelitinya hanya sedikit 
    return data, sheet

def preprocessing(data, listtoberemoved): #memulai proses preprocessing
    df = pd.DataFrame(data['Judul'],columns=['Judul']).copy() #membuat data frame awal yang isi nya judul
    cleaned = [] #membuat list baru berjudul cleaned
    for n in df['Judul'].values:
        n = n.lower() #proses case folding
        n = re.sub(r':', '', n) #proses cleaning
        n = re.sub(r'‚Ä¶', '', n) #proses cleaning
        n = re.sub(r'[^\x00-\x7F]+',' ', n) #proses cleaning
        n = re.sub('[^a-zA-Z]', ' ', n) #proses cleaning
        n = re.sub("&lt;/?.*?&gt;","&lt;&gt;",n) #proses cleaning
        n = re.sub("(\\d|\\W)+"," ",n) #proses cleaning
        n = re.sub(r'â', '', n) #proses cleaning
        n = re.sub(r'€', '', n) #proses cleaning
        n = re.sub(r'¦', '', n) #proses cleaning
        cleaned.append(n) #setelah dicleaning, data ditambahkan dilist yg nama nya cleaning
    df['cleaned'] = cleaned  #setelah list cleaning terbentuk, maka membuat satu kolom didata frame df yg nama kolomnya cleaned, dan data clean akan dimasukkan

    tokenized = [] #memulai list baru berjudul token
    for n in cleaned: #mentokenize data yang berasal dari proses cleaning
        n = word_tokenize(n) 
        tokenized.append(n)
    df['tokenized'] = [', '.join(n) for n in tokenized] #dan hasil tokenize dimasukkan pada kolom yang baru bernama tokinezed

    removed = [] #memulai list baru berjudul removed
    for ts in tokenized: #menremoved data stopwords dari proses tokenized
        n = []
        for t in ts:
            if t not in listtoberemoved and t not in listStopword and t not in string.punctuation: #selain meremove stopwords, listoberemoved yg sebelumnya diatur jga dihapus
                n.append(t)
        removed.append(n)
    df['removed'] = [', '.join(n) for n in removed] #hasil removed dimasukkan ke kolom baru bernama removed

    stemmed = [] #memulai list baru berjudul stemmed
    for n in removed: #proses stemming dilakukan dari data removed
        n = ' '.join(n)
        n = stemmer.stem(n)
        n = n.split(' ')
        stemmed.append(n)
    df['stemmed'] = [' '.join(n) for n in stemmed] #hasil stemming dimasukkan ke kolom baru bernama stemmed
    return df, stemmed

def mining(data,stemmed, ms, mp, mnp): #mulai data mining

    dx = [n for n in [a + b + c for a,b,c in zip(stemmed,data['Keyword'].str.split(",").values.tolist(),data['Topik'].str.split(",").values.tolist())]] #hasil dari proses stemming, digabungkan dengan keyword dan topik, sehingga pattern nya nambah
    ps = PrefixSpan(dx) #data sebelumnya, dimasukkan ke proses prefixspan. untuk mendapatkan prefixspan diatur min freq=2(yg pattern muncul hanya sekali, tidak dihitung), max pattern=10(yg pattern>10 tidak akan muncul), min pattern=2(<2 tidak dihitung)
    pf_results = pd.DataFrame(ps.frequent(ms), columns=['freq','sequence']) #untuk setting min freq
    
    pf_results['sequence'] = [', '.join(n) for n in pf_results['sequence'].values.tolist()] #untuk menggabungkan list pd proses preprocessing, karena sebelumnya hanya breket atau tanda kurung menjadi tabel
    pf_results = pf_results[[len(n)<=mp for n in pf_results['sequence'].str.split(",").values.tolist()]] #memfilter max patt
    pf_results = pf_results[[len(n)>=mnp for n in pf_results['sequence'].str.split(",").values.tolist()]].sort_values(by='freq',ascending=False).reset_index(drop=True) #memfilter min patt
    return pf_results #memunculkan hasil 

def run(seq,ms,mp,mnp): #merunning fungsi diatas
    data, sheet= read_data(seq)
    df, stemmed = preprocessing(data, listtoberemoved)
    pf = mining(data, stemmed, ms=ms, mp=mp, mnp=mnp)
    return pf

In [ ]:
memory_limit()

In [ ]:
report = []
for i,jur in enumerate(["Informatika","Sipil","Mesin","Elektro","Arsitektur","Sistem Informasi"]):
    for j in range(2,7):
        start_time = time.time()
        if jur=="Mesin" and j == 2:
            continue
        else:
            try:
                run(i+1,j,10,3)
            except MemoryError:
                print("Memory Insufficient")
                continue
        # print(jur,j,time.time() - start_time, get_memory())
        report.append([jur,j,time.time() - start_time, get_memory()])

df_report = pd.DataFrame(report, columns=["data","min support","running time (s)", "memory usage (kB)"])

In [ ]:
df_report[["data","min support","running time (s)"]][df_report["data"]=="Mesin"].set_index("min support").plot.bar(rot=0, subplots=True)
plt.show()

In [ ]:
min(df_report["running time (s)"])

In [ ]:
for h in ["running time (s)","memory usage (kB)"]:
    fig, axs = plt.subplots(2, 3,figsize = (20,15), sharex=True)
    for i,jur in enumerate(["Informatika","Sipil","Mesin","Elektro","Arsitektur","Sistem Informasi"]):
        axs[i//3, i%3].bar(
            df_report[["data","min support",h]][df_report["data"]==jur].set_index("min support").index.astype(str), 
            df_report[["data","min support",h]][df_report["data"]==jur].set_index("min support")[h].values
        )
        axs[i//3, i%3].set_title(jur)
        axs[i//3, i%3].set_ylim([min(df_report[h]), max(df_report[h]) + max(df_report[h])*0.01])

    for ax in axs.flat:
        ax.set(xlabel='minimum support', ylabel=h)

    for ax in axs.flat:
        ax.label_outer()
    
    fig.suptitle(f'Prefix Span Performance based on Min. Support & {h}', fontsize=16, y=0.92)
    fig.show()

In [ ]:
for h in ["running time (s)","memory usage (kB)"]:
    fig, axs = plt.subplots(2, 3,figsize = (20,15), sharex=True)
    fig = plt.figure(figsize = (8,6))
    for i,jur in enumerate(["Informatika","Sipil","Mesin","Elektro","Arsitektur","Sistem Informasi"]):
        plt.plot(
            df_report[["data","min support",h]][df_report["data"]==jur].set_index("min support").index.astype(str), 
            df_report[["data","min support",h]][df_report["data"]==jur].set_index("min support")[h].values,
            label = jur
        )
        axs[i//3, i%3].set_title(jur)
        axs[i//3, i%3].set_ylim([min(df_report[h]), max(df_report[h]) + max(df_report[h])*0.01])

    for ax in axs.flat:
        ax.set(xlabel='minimum support', ylabel=h)

    for ax in axs.flat:
        ax.label_outer()
    plt.legend(loc="center right")
    fig.suptitle(f'Prefix Span Performance based on Min. Support & {h}', fontsize=12, y=0.95)
    fig.show(marker='s')